# Motion Correction Tutorial

This Jupyter Notebook presents an example of the application of motion correction in T1 and DWI MRI images.


The motion correction algorithm uses Model-Driven Registration (MDR). `ukat` calls the python package `mdr_library` and its source code can be found at: https://github.com/QIB-Sheffield/MDR_Library


Start by importing the required libraries and defining some settings:

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from ukat.data import fetch
from ukat.utils.tools import convert_to_pi_range
from ukat.mapping.t1 import T1, magnitude_correct
from ukat.mapping.diffusion import ADC, DTI
from ukat.moco.mdr import MotionCorrection

# Ensure figures are rendered in the notebook
%matplotlib inline

# Initialise output path for the Model-Driven Registration process
directory = 'motion_correction_output'
os.makedirs(directory, exist_ok=True)
OUTPUT_DIR = os.path.join(os.getcwd(), directory)

Fetch T1 test data

In [2]:
# We will process on a central slice only
magnitude, phase, affine_t1, ti, tss = fetch.t1_philips(2)
magnitude = magnitude[:, :, 3, :]
phase = phase[:, :, 3, :]
# Prepare the T1 modelling input arguments
phase = convert_to_pi_range(phase)
complex_data = magnitude * (np.cos(phase) + 1j * np.sin(phase)) # convert magnitude and phase into complex data
ti = np.array(ti) * 1000  # convert TIs to ms
magnitude_corrected = np.squeeze(magnitude_correct(complex_data))
# Pre-processing as preparation for the Model-Driven Registration process
list_input_parameters_t1 = [affine_t1, ti, 0, None, None, 2, False]

invalid value encountered in true_divide
invalid value encountered in true_divide
invalid value encountered in log


Fetch DWI test data

In [3]:
pixel_array, affine_dwi, bvals, bvecs = fetch.dwi_philips()
# Pre-processing as preparation for the Model-Driven Registration process
list_input_parameters_dwi = [affine_dwi, bvals, None, False]

Set masks for motion correction

In [9]:
mask_flag = True
if mask_flag == True:
    x_t1 = np.shape(magnitude_corrected)[0]
    y_t1 = np.shape(magnitude_corrected)[1]
    x_dwi = np.shape(pixel_array)[0]
    y_dwi = np.shape(pixel_array)[1]
    mask_moco_t1 = np.ones(np.shape(magnitude_corrected))
    mask_moco_dwi = np.ones(np.shape(pixel_array))
    mask_moco_t1[:int(x_t1/4), ...] = 0
    mask_moco_t1[int(3*x_t1/4):, ...] = 0
    mask_moco_t1[:, :int(y_t1/4), ...] = 0
    mask_moco_t1[:, int(3*y_t1/4):, ...] = 0
    mask_moco_dwi[:int(x_dwi/4), ...] = 0
    mask_moco_dwi[int(3*x_dwi/4):, ...] = 0
    mask_moco_dwi[:, :int(y_dwi/4), ...] = 0
    mask_moco_dwi[:, int(3*y_dwi/4):, ...] = 0
    mask_t1_model = np.array(mask_moco_t1[..., 3], dtype=bool)
    mask_dwi_model = np.array(mask_moco_dwi[..., 3, 0], dtype=bool)
    list_input_parameters_t1 = [affine_t1, ti, 0, None, mask_t1_model, 2, False]
    list_input_parameters_dwi = [affine_dwi, bvals, mask_dwi_model, False]
else:
    mask_moco_t1 = None
    mask_moco_dwi = None

## T1 Model-Driven Registration

In [ ]:
t1_registration = MotionCorrection(magnitude_corrected, affine_t1, 'T1_Moco', list_input_parameters_t1, convergence=1, multithread=False, log=False, mask=mask_moco_t1)

In [ ]:
# Get the T1 coregistered pixel_array and the array with the difference between the original and the motion corrected
t1_moco_array = t1_registration.get_coregistered()
t1_moco_diff_array = t1_registration.get_diff_orig_coreg()
# Save motion corrected T1 sequence to NIfTI
t1_registration.to_nifti(output_directory=OUTPUT_DIR, base_file_name='T1_motion_corrected', maps=['original', 'coregistered', 'difference', 'fitted', 'deformation_field', 'params'])

## DWI Model-driven registration

In [10]:
dwi_registration = MotionCorrection(pixel_array, affine_dwi, 'DWI_Moco', list_input_parameters_dwi, convergence=1, multithread=True, log=False, mask=mask_moco_dwi)

(128, 128)
(128, 128, 79)


Co-registration progress: 100%|██████████| 79/79 [02:57<00:00,  2.24s/it]


(128, 128)
(128, 128, 79)


Co-registration progress: 100%|██████████| 79/79 [02:50<00:00,  2.16s/it]


(128, 128)
(128, 128, 79)


...

Co-registration progress: 100%|██████████| 79/79 [02:56<00:00,  2.24s/it]


(128, 128)
(128, 128, 79)


Co-registration progress: 100%|██████████| 79/79 [02:59<00:00,  2.28s/it]


(128, 128)
(128, 128, 79)


Co-registration progress:   0%|          | 0/79 [00:00<?, ?it/s]

In [ ]:
# Get the DWI coregistered pixel_array and the array with the difference between the original and the motion corrected
dwi_moco_array = dwi_registration.get_coregistered()
dwi_moco_diff_array = dwi_registration.get_diff_orig_coreg()
# Save motion corrected diffusion sequence to NIfTI
dwi_registration.to_nifti(output_directory=OUTPUT_DIR, base_file_name='DWI_motion_corrected', maps=['original', 'coregistered', 'difference', 'fitted', 'deformation_field', 'params'])

In [ ]:
# Calculate maps from the original T1 sequence save as niftis
t1_mapper = T1(magnitude_corrected, ti, affine=affine_t1, multithread=True, parameters=2)
t1_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='T1_original', maps='all')

# Calculate maps from the motion corrected T1 sequence and save as niftis
t1_moco_mapper = T1(t1_moco_array, ti, affine=affine_t1, multithread=True, parameters=2)
t1_moco_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='T1_moco', maps='all')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

# Display the T1 map generated from the original data
im = ax1.imshow(np.rot90(t1_mapper.t1_map), cmap='inferno', clim=(250, 2250))
cb = fig.colorbar(im, ax=ax1)
cb.set_label('$T_1 Original$ (ms)')
ax1.axis('off')

# Display the T1 map generated from the motion corrected data
im2 = ax2.imshow(np.rot90(t1_moco_mapper.t1_map), cmap='inferno', clim=(250, 2250))
cb = fig.colorbar(im2, ax=ax2)
cb.set_label('$T_1 Moco$ (ms)')
ax2.axis('off')

plt.show()

In [ ]:
# Generate a mask based on the intensity of the b0 volume. This will reduce computation times.
mask = pixel_array[..., 0] > 20000

# Calculate maps from the original diffusion sequence using ADC methods and save as niftis
adc_mapper = ADC(pixel_array, affine_dwi, bvals, mask=mask, ukrin_b=True)
adc_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_original', maps='all')

# Calculate maps from the original diffusion sequence using DTI methods and save as niftis
dti_mapper = DTI(pixel_array, affine_dwi, bvals, bvecs, mask=mask, ukrin_b=True)
dti_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_original', maps=['md', 'fa'])

# Calculate maps from the motion corrected diffusion sequence using ADC methods and save as niftis
adc_mapper_moco = ADC(dwi_moco_array, affine_dwi, bvals, mask=mask, ukrin_b=True)
adc_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_moco', maps='all')

# Calculate maps from the motion corrected diffusion sequence using DTI methods and save as niftis
dti_mapper_moco = DTI(dwi_moco_array, affine_dwi, bvals, bvecs, mask=mask, ukrin_b=True)
dti_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_moco', maps=['md', 'fa'])

In [ ]:
# Display the central slice of each map
fig, ax = plt.subplots(2, 2, figsize=(8, 6))

# Display a central slice of the ADC map
im = ax[0, 0].imshow(np.rot90(adc_mapper.adc[:, :, adc_mapper.shape[2]//2]), cmap='inferno', clim=(0.001, 0.003))
cb = fig.colorbar(im, ax=ax[0, 0])
cb.set_label('ADC ($mm^2/s$)')
ax[0, 0].axis('off')

# Display a central slice of the MD map
im = ax[0, 1].imshow(np.rot90(dti_mapper.md[:, :, dti_mapper.shape[2]//2]), cmap='inferno', clim=(0.001, 0.003))
cb = fig.colorbar(im, ax=ax[0, 1])
cb.set_label('MD ($mm^2/s$)')
ax[0, 1].axis('off')

# Display a central slice of the FA map
im = ax[1, 0].imshow(np.rot90(dti_mapper.fa[:, :, dti_mapper.shape[2]//2]), cmap='viridis', clim=(0.1, 0.8))
cb = fig.colorbar(im, ax=ax[1, 0])
cb.set_label('FA')
ax[1, 0].axis('off')

# Display a central slice of the color FA map
im = ax[1, 1].imshow(np.rot90(dti_mapper.color_fa[:, :, dti_mapper.shape[2]//2, :]))
ax[1, 1].axis('off')

In [ ]:
# Display the central slice of each map
fig, ax = plt.subplots(2, 2, figsize=(8, 6))

# Display a central slice of the ADC map
im = ax[0, 0].imshow(np.rot90(adc_mapper_moco.adc[:, :, adc_mapper_moco.shape[2]//2]), cmap='inferno', clim=(0.001, 0.003))
cb = fig.colorbar(im, ax=ax[0, 0])
cb.set_label('ADC ($mm^2/s$)')
ax[0, 0].axis('off')

# Display a central slice of the MD map
im = ax[0, 1].imshow(np.rot90(dti_mapper_moco.md[:, :, dti_mapper_moco.shape[2]//2]), cmap='inferno', clim=(0.001, 0.003))
cb = fig.colorbar(im, ax=ax[0, 1])
cb.set_label('MD ($mm^2/s$)')
ax[0, 1].axis('off')

# Display a central slice of the FA map
im = ax[1, 0].imshow(np.rot90(dti_mapper_moco.fa[:, :, dti_mapper_moco.shape[2]//2]), cmap='viridis', clim=(0.1, 0.8))
cb = fig.colorbar(im, ax=ax[1, 0])
cb.set_label('FA')
ax[1, 0].axis('off')

# Display a central slice of the color FA map
im = ax[1, 1].imshow(np.rot90(dti_mapper_moco.color_fa[:, :, dti_mapper_moco.shape[2]//2, :]))
ax[1, 1].axis('off')